# TP3 : Attention

In [ ]:
import math
import torch

import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Attention


In [ ]:
# Input size : b, T, dk
# Output size : b, T, dv
class SelfAttentionCustom(nn.Module):
    def __init__(self, dk=10, dv=10):
        super().__init__()
        self.dk = dk
        self.dv = dv

        # Define the linear layers to project input to keys, queries, and values
        # YOUR CODE HERE
        # self.to_keys = nn.Linear(???, ???, bias=False)
        # self.to_queries = nn.Linear(???, ???, bias=False)
        # self.to_values = nn.Linear(???, ???, bias=False)

    def forward(self, xs):
        K = self.to_keys(xs)  # b, T,  dk
        Q = self.to_queries(xs)  # b, T,  dk
        V = self.to_values(xs)  # b, T,  dv

        out = None
        # YOUR CODE HERE
        return out

In [ ]:
attention_custom = SelfAttentionCustom(dk=10, dv=10)

x = torch.randn(5, 3, 10)  # b, T, k
print(attention_custom(x))

# Multi-head Attention

### Start naive with a for loop

Use torch.cat to concatenate together a list of tensors on a given dimension.

In [ ]:
class MultiHeadAttentionCustom(nn.Module):
    def __init__(self, h=5, dk=10, dv=10):
        super().__init__()
        self.dk = dk
        self.dv = dv
        self.h = h

        self.to_keys = nn.Linear(dk, dk, bias=False)
        self.to_queries = nn.Linear(dk, dk, bias=False)
        self.to_values = nn.Linear(dk, dv, bias=False)

        self.unifyhead = nn.Linear(dv, dv)

    def forward(self, x):
        h, dk, dv = self.h, self.dk, self.dv
        dkh, dvh = dk // h, dv // h  # dimension per head

        K = self.to_keys(x)  # b, T, dk
        Q = self.to_queries(x)
        V = self.to_values(x)

        # YOUR CODE HERE
        out = None
        return out


x = torch.randn(2, 10, 32)  # b, T, k
block = MultiHeadAttentionCustom(h=4, dk=32, dv=16)
out = block(x)
print(out.shape)

### Vectorized

To change the shape of a tensor, use the view function. 
For example:

In [ ]:
x = torch.randn(2, 4)  # b, T, k
print(x)
x = x.view(2, 2, 2)
print(x)

In [ ]:
# Input size : b, T, dk
# Output size : b, T, dv
class MultiHeadAttentionVectorized(nn.Module):
    def __init__(self, h=5, dk=10, dv=10):
        super().__init__()
        self.d_k = dk
        self.dv = dv
        self.h = h

        self.to_keys = nn.Linear(dk, dk, bias=False)
        self.to_queries = nn.Linear(dk, dk, bias=False)
        self.to_values = nn.Linear(dk, dv, bias=False)

        self.unifyhead = nn.Linear(dv, dv)

    def forward(self, x):
        # Input size : b, T, dk
        # Output size : b, T, dv
        b, T, dk = x.shape
        h, dv = self.h, self.dv

        # Get queries, keys, values
        Q = self.to_queries(x)  # (b, T, dk)
        K = self.to_keys(x)
        V = self.to_values(x)

        # YOUR CODE HERE
        out = None
        return out


In [ ]:
x = torch.randn(2, 10, 32)  # b, T, k
MHAVect = MultiHeadAttentionVectorized(h=4, dk=32, dv=16)
MHACustom = MultiHeadAttentionCustom(h=4, dk=32, dv=16)

# Since both models have the same architecture, we can copy the weights
MHAVect.load_state_dict(MHACustom.state_dict())

out_vect = MHAVect(x)
out_custom = MHACustom(x)
print(out_vect - out_custom)  # should be all zeros


In [ ]:
%timeit MHACustom(x)
%timeit MHAVect(x)

# Faster
We can even use torch.bmm for (marginally) faster implementation. 

torch.bmm is highly optimized matrix multiplication specifically 
for inputs of shape (b, n, m), (b, m, k) 

In [ ]:
# Input size : b, T, dk
# Output size : b, T, dv
class MultiHeadAttentionBMM(nn.Module):
    def __init__(self, h=5, dk=10, dv=10):
        super().__init__()
        self.dk = dk
        self.dv = dv
        self.h = h

        self.to_keys = nn.Linear(dk, dk, bias=False)
        self.to_queries = nn.Linear(dk, dk, bias=False)
        self.to_values = nn.Linear(dk, dv, bias=False)

        self.unifyhead = nn.Linear(dv, dv)

    def forward(self, x):
        # Input:  (b, T, dk)
        # Output: (b, T, dv)
        b, T, dk = x.shape
        h, dv = self.h, self.dv
        dh = dk // h

        # project
        Q = self.to_queries(x).view(b, T, h, dh).transpose(1, 2)  # b, h, T, dh
        K = self.to_keys(x).view(b, T, h, dh).transpose(1, 2)  # b, h, T, dh
        V = self.to_values(x).view(b, T, h, dv // h).transpose(1, 2)  # b, h, T, dv // h

        # YOUR CODE HERE
        out = None
        return out


### Check your results

In [ ]:
x = torch.randn(32, 10, 32)  # b, T, k
MHACustom = MultiHeadAttentionCustom(h=4, dk=32, dv=16)
MHAVect = MultiHeadAttentionVectorized(h=4, dk=32, dv=16)
MHABmm = MultiHeadAttentionBMM(h=4, dk=32, dv=16)

# Since both models have the same architecture, we can copy the weights
MHAVect.load_state_dict(MHACustom.state_dict())
MHABmm.load_state_dict(MHACustom.state_dict())

out_custom = MHACustom(x)
out_vect = MHAVect(x)
out_BMM = MHABmm(x)
# print(out_vect - out_custom)  # should be all zeros
print(out_vect - out_BMM)  # should be all zeros

In [ ]:
%timeit MHACustom(x)
%timeit MHAVect(x)
%timeit MHABmm(x)

# Add Masking

Apply causal masking to prevent tokens from attending future tokens.

Use the torch.tril (or torch.triu) function to build a triangular mask that will be applied to to the attention scores (see lecture slides).

Use the 'masked_fill' function to apply the mask to the attention scores matrix. 

In [ ]:
class MultiHeadAttentionCausal(nn.Module):
    def __init__(self, h=5, dk=10, dv=10):
        super().__init__()
        self.d_k = dk
        self.dv = dv
        self.h = h

        self.to_keys = nn.Linear(dk, dk, bias=False)
        self.to_queries = nn.Linear(dk, dk, bias=False)
        self.to_values = nn.Linear(dk, dv, bias=False)

        self.unifyhead = nn.Linear(dv, dv)

    def forward(self, x):
        # Input:  (b, T, dk)
        # Output: (b, T, dv)
        b, T, dk = x.shape
        h, dv = self.h, self.dv
        dh = dk // h

        # project
        Q = self.to_queries(x).view(b, T, h, dh).transpose(1, 2)  # (b, h, T, dh)
        K = self.to_keys(x).view(b, T, h, dh).transpose(1, 2)  # (b, h, T, dh)
        V = (
            self.to_values(x).view(b, T, h, dv // h).transpose(1, 2)
        )  # (b, h, T, dv // h)

        # reshape for bmm: merge batch and head dims
        Q = Q.reshape(b * h, T, dh)
        K = K.reshape(b * h, T, dh)
        V = V.reshape(b * h, T, dv // h)

        # scaled dot-product attention
        # YOUR CODE HEREh
        scores = None

        attn = torch.softmax(scores, dim=-1)
        out = torch.bmm(attn, V)  # (b*h, T, dv // h)

        # reshape back and unify heads
        out = out.view(b, h, T, dv // h).transpose(1, 2).reshape(b, T, dv)
        return self.unifyhead(out)


# Normalization Layer

### Layer Normalization (formula)

$$
\operatorname{LayerNorm}(x) 
= \gamma \odot \frac{x - \mu}{\sqrt{\sigma^2 + \varepsilon}} + \beta
$$

where  

$$
\mu = \frac{1}{d} \sum_{i=1}^d x_i,
\quad
\sigma^2 = \frac{1}{d} \sum_{i=1}^d (x_i - \mu)^2,
$$

and normalization is done over the last dimension (of size \(d\)).

In [ ]:
class SimpleLayerNorm(nn.Module):
    def __init__(self, dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.gamma = nn.Parameter(torch.ones(dim))
        self.beta = nn.Parameter(torch.zeros(dim))

    def forward(self, x):
        # YOUR CODE HERE
        return

# Full Block
Apply the diagram


<img src="images/transformer_block.png" width="200">

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, dk, heads):
        super().__init__()

        self.attention = MultiHeadAttentionCausal(heads, dk, dk)
        self.norm1 = SimpleLayerNorm(dk)
        self.norm2 = SimpleLayerNorm(dk)
        self.ff = nn.Sequential(nn.Linear(dk, 4 * dk), nn.GELU(), nn.Linear(4 * dk, dk))

    def forward(self, x):
        # YOUR CODE HERE
        return


x = torch.randn(2, 10, 32)  # b, T, k
block = TransformerBlock(dk=32, heads=4)
out = block(x)
print(out.shape)

# Positional Encoding

Remember slicing: 
- arr[0::2] takes the even indices (starts at 0 and skips with a stride of 2)
- arr[0::2] takes the even indices (starts at 1 and skips with a stride of 2)

Formula
$$
	p_t^{(2i)} = \sin\!\Big(\frac{t}{10000^{2i/d}}\Big),
	\quad
	p_t^{(2i+1)} = \cos\!\Big(\frac{t}{10000^{2i/d}}\Big)
$$

In [ ]:
def positional_encoding(seq_len, dk):
    # T, d_K
    # YOUR CODE HERE
    return


pe = positional_encoding(100, 64)  # T, d_K

plt.figure(figsize=(10, 6))
plt.imshow(pe.T, aspect="auto", cmap="viridis", origin="lower")
plt.xlabel("Position")
plt.ylabel("Dimension")
plt.title("Sinusoidal Positional Encoding")
plt.colorbar(label="Value")
plt.show()


# A Full Model

Stack multiple Transformer blocks and add a prediction head that maps the embeddings 
to logits over the vocabulary.

We now use some more standard design choices: 
- tokens are represented as indices (integers), we recover their embeddings by applying an embedding layer. We assume some preproprocessing was done on the data to convert  tokens into their token indices. 
- we overload the forward method:
    - when training, forward may take targets as inputs in addition to the token indices. The loss is computed inside forward and returned. 

In [ ]:
class TinyTransformer(nn.Module):
    def __init__(self, vocab_size, h, dk, n_layers, block_size, device="cpu"):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, dk)
        self.pos_emb = positional_encoding(block_size, dk)
        self.blocks = nn.Sequential(*[TransformerBlock(dk, h) for _ in range(n_layers)])
        self.ln_f = SimpleLayerNorm(dk)

        self.prediction_head = nn.Linear(dk, vocab_size)

        self.device = device
        self.vocab_size = vocab_size
        self.block_size = block_size

    def forward(self, idx, targets=None):
        B, T = idx.shape
        pos = torch.arange(T, device=self.device)
        # YOUR CODE HERE
        logits = None

        if targets is None:
            return logits, None
        loss = F.cross_entropy(logits.view(-1, self.vocab_size), targets.view(-1))
        return logits, loss

# Next-token prediction

We are now ready to train a transformer to generate text. 

Check the input.txt file in the folder. It contains Shakespeare pieces. 

### Preprocessing:
We split the text into characters and encodes the individual characters into 
unique indices.


### Fill the get_batch function

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

text = open("data/input.txt", "r").read()
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}


def encode(s):
    return torch.tensor([stoi[c] for c in s], dtype=torch.long)


def decode(t):
    return "".join([itos[int(i)] for i in t])


data = encode(text)
split = int(0.9 * len(data))
train_data, val_data = data[:split], data[split:]


def get_batch(src, batch_size, block_size, device="cpu"):
    # Picks a random batch of data from the source tensor
    # Returns input and target tensors of shape (batch_size, block_size)
    ix = torch.randint(len(src) - block_size, (batch_size,))

    # YOUR CODE HERE

    x, y = None, None
    return x.to(device), y.to(device)

In [ ]:
print("Vocabulary size:", vocab_size)
print("Number of training tokens:", len(train_data))

In [ ]:
h = 4
n_layers = 2
dk = 128
block_size = 128
batch_size = 32

model = TinyTransformer(vocab_size, h, dk, n_layers, block_size, device=device).to(
    device
)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


n_params = count_parameters(model)
print(f"Trainable parameters: {n_params}")

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
losses = []
for step in range(2000):
    xb, yb = get_batch(train_data, batch_size, block_size)

    # YOUR CODE HERE

    loss = None
    losses.append(loss.item())
    if step % 200 == 0:
        print(f"step {step}: loss {loss.item():.4f}")


# Generate some text

use the torch.multinomial function to sample a sequence.

In [ ]:
def generate(model, idx, max_new_tokens):
    # Generate new tokens given a context idx
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -model.block_size :]  # crop context to block size
        logits, _ = model(idx_cond)
        # YOUR CODE HERE
        next_token = None
        idx = torch.cat((idx, next_token), dim=1)
    return idx

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(generate(model, context, max_new_tokens=500)[0].tolist()))